# Bedrock - Getting started with the python API

- Documentation Root: [Bedrock Documentation Root](https://docs.aws.amazon.com/bedrock/)
- Quickstart: https://docs.aws.amazon.com/bedrock/latest/userguide/getting-started-api.html 

### Requirements:
- AWS IAM user
- IAM user access key
- awscli installed
- awscli configured

Obs: Check the quickstart from the user-guide for instructions on how to set up the pre-requisites.

---

## Attach Amazon Bedrock permissions to a user or role

The permission json that needs to be attache. Save it into a input.json file:

```json
{
  "Version": "2012-10-17",
  "Statement": [
      {
          "Sid": "MarketplaceBedrock",
          "Effect": "Allow",
          "Action": [
              "aws-marketplace:ViewSubscriptions",
              "aws-marketplace:Unsubscribe",
              "aws-marketplace:Subscribe"
          ],
          "Resource": "*"
      }
  ]
}
```

### Using the console

After setting up credentials for programmatic access, you need to configure permissions for a user or IAM role to have access a set of Amazon Bedrock-related actions. To set up these permissions, do the following:

1. On the AWS Management Console Home page, select the IAM service or navigate to the IAM console at https://console.aws.amazon.com/iam/

2. Select Users or Roles and then select your user or role.

3. In the Permissions tab, choose Add permissions and then choose Add AWS managed policy. Choose the AmazonBedrockFullAccess AWS managed policy.

4. To allow the user or role to subscribe to models, choose Create inline policy and then specify the permissions json file above in the JSON editor.

### Using awscli

#### Step 2: Attach Managed Policy (`AmazonBedrockFullAccess`) to the IAM User or Role

Use the following AWS CLI command to attach the `AmazonBedrockFullAccess` policy to your IAM user or role.

##### For an IAM User:
```bash
aws iam attach-user-policy --user-name YOUR_USER_NAME --policy-arn arn:aws:iam::aws:policy/AmazonBedrockFullAccess
```

Replace `YOUR_USER_NAME` with the name of the IAM user to which you want to attach the policy.

##### For an IAM Role:
```bash
aws iam attach-role-policy --role-name YOUR_ROLE_NAME --policy-arn arn:aws:iam::aws:policy/AmazonBedrockFullAccess
```

Replace `YOUR_ROLE_NAME` with the name of the IAM role.

#### Step 3: Create an Inline Policy for Subscription Permissions

To allow the user or role to subscribe to models, you'll need to create an inline policy that grants the necessary permissions (`aws-marketplace:ViewSubscriptions`, `aws-marketplace:Unsubscribe`, and `aws-marketplace:Subscribe`).

##### For an IAM User:
```bash
aws iam put-user-policy --user-name YOUR_USER_NAME --policy-name MarketplaceBedrockInlinePolicy --policy-document 'file://input.json'
```

##### For an IAM Role:
```bash
aws iam put-role-policy --role-name YOUR_ROLE_NAME --policy-name MarketplaceBedrockInlinePolicy --policy-document 'file://input.json'
```

Replace `YOUR_USER_NAME` or `YOUR_ROLE_NAME` with the appropriate IAM user or role name and the correct path to the json file.

#### Step 4: Verify Permissions

To verify the permissions are successfully attached to the IAM user or role, you can use the following commands:

##### For an IAM User:
```bash
aws iam list-attached-user-policies --user-name YOUR_USER_NAME
aws iam list-user-policies --user-name YOUR_USER_NAME
```

##### For an IAM Role:
```bash
aws iam list-attached-role-policies --role-name YOUR_ROLE_NAME
aws iam list-role-policies --role-name YOUR_ROLE_NAME
```

These commands will list the attached managed and inline policies for the user or role.

---

## Request access to Amazon Bedrock models

Request access to the Amazon Bedrock models through the Amazon Bedrock console by following the steps at [Request access to an Amazon Bedrock foundation model](https://docs.aws.amazon.com/bedrock/latest/userguide/getting-started.html#getting-started-model-access).

---

#### **Step 3: List Available Models**
Check the foundational models (FMs) available through AWS Bedrock:

In [2]:
import boto3

# Create an Amazon Bedrock Runtime client.
bedrock = boto3.client("bedrock")

# List models
response = bedrock.list_foundation_models()
models = response.get("modelSummaries", [])

import pandas as pd
pd.DataFrame(models).head()

,modelArn,modelId,modelName,providerName,inputModalities,outputModalities,responseStreamingSupported,customizationsSupported,inferenceTypesSupported,modelLifecycle
0,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-tg1-large,Titan Text Large,Amazon,[TEXT],[TEXT],True,[],[ON_DEMAND],{'status': 'ACTIVE'}
1,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-image-generator-v1:0,Titan Image Generator G1,Amazon,"[TEXT, IMAGE]",[IMAGE],NaN,[FINE_TUNING],[PROVISIONED],{'status': 'ACTIVE'}
2,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-image-generator-v1,Titan Image Generator G1,Amazon,"[TEXT, IMAGE]",[IMAGE],NaN,[],[ON_DEMAND],{'status': 'ACTIVE'}
3,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-image-generator-v2:0,Titan Image Generator G1 v2,Amazon,"[TEXT, IMAGE]",[IMAGE],NaN,[FINE_TUNING],"[PROVISIONED, ON_DEMAND]",{'status': 'ACTIVE'}
4,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-text-premier-v1:0,Titan Text G1 - Premier,Amazon,[TEXT],[TEXT],True,[],[ON_DEMAND],{'status': 'ACTIVE'}


---

## Submit a text prompt to a model and generate a text response with InvokeModel

- Reference Documentaion: https://docs.aws.amazon.com/bedrock/latest/userguide/getting-started-api-ex-python.html

The following example runs the InvokeModel operation using an Amazon Bedrock client. InvokeModel lets you submit a prompt to generate a model response. Run the following SDK for Python script to create an Amazon Bedrock runtime client and generate a text response with the operation:

In [1]:
# Use the native inference API to send a text message to Amazon Titan Text G1 - Express.

import boto3
import json

from botocore.exceptions import ClientError

# Create an Amazon Bedrock Runtime client.
brt = boto3.client("bedrock-runtime")

# Set the model ID, e.g., Amazon Titan Text G1 - Express.
model_id = "amazon.titan-text-express-v1"

# Define the prompt for the model.
prompt = "Describe the purpose of a 'hello world' program in one line."

# Format the request payload using the model's native structure.
native_request = {
    "inputText": prompt,
    "textGenerationConfig": {
        "maxTokenCount": 512,
        "temperature": 0.5,
        "topP": 0.9
    },
}

# Convert the native request to JSON.
request = json.dumps(native_request)

try:
    # Invoke the model with the request.
    response = brt.invoke_model(modelId=model_id, body=request)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)

# Decode the response body.
model_response = json.loads(response["body"].read())

# Extract and print the response text.
response_text = model_response["results"][0]["outputText"]
print(response_text)


A 'hello world' program serves as a basic introduction to programming and demonstrates the fundamental syntax and structure of a programming language.


If the command is successful, the response returns the text generated by the model in response to the prompt.

---

## Submit a text prompt to a model and generate a text response with Converse

The following example runs the Converse operation using an Amazon Bedrock client. We recommend using Converse operation over InvokeModel when supported, because it unifies the inference request across Amazon Bedrock models and simplifies the management of multi-turn conversations. Run the following SDK for Python script to create an Amazon Bedrock runtime client and generate a text response with the Converse operation:

In [2]:
# Use the Conversation API to send a text message to Amazon Titan Text G1 - Express.

import boto3
from botocore.exceptions import ClientError

# Create an Amazon Bedrock Runtime client.
brt = boto3.client("bedrock-runtime")

# Set the model ID, e.g., Amazon Titan Text G1 - Express.
model_id = "amazon.titan-text-express-v1"

# Start a conversation with the user message.
user_message = "Describe the purpose of a 'hello world' program in one line."
conversation = [
    {
        "role": "user",
        "content": [{"text": user_message}],
    }
]

try:
    # Send the message to the model, using a basic inference configuration.
    response = brt.converse(
        modelId=model_id,
        messages=conversation,
        inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},
    )

    # Extract and print the response text.
    response_text = response["output"]["message"]["content"][0]["text"]
    print(response_text)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)



A 'hello world' program in one line is a basic computer program that outputs the phrase "hello world" to the console. It serves as a fundamental example of how to write and execute a program in a particular programming language.


If the command is successful, the response returns the text generated by the model in response to the prompt.